In [36]:
import os

In [37]:
os.chdir("E:\Weather_prediction")

In [38]:
%pwd

'E:\\Weather_prediction'

In [39]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [40]:
from src.Predict_weather.constants import *
from src.Predict_weather.utils.common import read_yaml, create_directories

In [41]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH, schema_filepath=SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
        
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        create_directories([config.root_dir])
        
        data_transformation_config = DataTransformationConfig(
            root_dir= config.root_dir,
            data_path= config.data_path
        )
        
        
        return data_transformation_config

In [42]:
import os
from src.Predict_weather import logger
from sklearn.model_selection import train_test_split
import pandas as pd

In [43]:
# Debug: Check the actual structure of the weather data
data_path = "artifacts/data_ingestion/weather.csv"
data = pd.read_csv(data_path)
print("Dataset shape:", data.shape)
print("\nColumn names:")
print(list(data.columns))
print("\nFirst few rows:")
print(data.head())

Dataset shape: (1827, 19)

Column names:
['date', 'temperature_2m_max', 'temperature_2m_min', 'precipitation_sum', 'rain_sum', 'precipitation_hours', 'temperature_2m_mean', 'apparent_temperature_mean', 'cloud_cover_mean', 'cloud_cover_max', 'cloud_cover_min', 'sunshine_duration', 'wind_speed_10m_max', 'wind_gusts_10m_max', 'wind_gusts_10m_mean', 'wind_speed_10m_mean', 'relative_humidity_2m_max', 'relative_humidity_2m_min', 'relative_humidity_2m_mean']

First few rows:
                        date  temperature_2m_max  temperature_2m_min  \
0  2020-08-14 18:30:00+00:00              27.244              22.144   
1  2020-08-15 18:30:00+00:00              27.794              22.194   
2  2020-08-16 18:30:00+00:00              23.994              22.544   
3  2020-08-17 18:30:00+00:00              24.444              22.394   
4  2020-08-18 18:30:00+00:00              27.344              22.144   

   precipitation_sum   rain_sum  precipitation_hours  temperature_2m_mean  \
0           6.900

In [44]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        
    
    def transform_data(self):
        
        data = pd.read_csv(self.config.data_path)
        
        # Ensure the 'date' column exists before processing
        if 'date' not in data.columns:
            logger.error(f"Column 'date' not found in the dataset. Available columns: {list(data.columns)}")
            raise KeyError("Column 'date' not found in the dataset.")
        
        # Extract day, year, and month from the 'date' column
        data['day'] = pd.to_datetime(data['date'], errors='coerce').dt.day
        data['year'] = pd.to_datetime(data['date'], errors='coerce').dt.year
        data['month'] = pd.to_datetime(data['date'], errors='coerce').dt.month
        
        # Remove the original date column after extracting temporal features
        data = data.drop('date', axis=1)
        
        train, test = train_test_split(data, test_size=0.2, random_state=42)
        
        train.to_csv(os.path.join(self.config.root_dir, "train.csv"), index=False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"), index=False)
        
        logger.info("Splitted data into training and test sets")
        logger.info("Removed original date column after extracting temporal features")
        logger.info(f"Train shape: {train.shape}")
        logger.info(f"Test shape: {test.shape}")
        
        print(f"Train shape: {train.shape}")            
        print(f"Test shape: {test.shape}")
        print("Date column removed successfully!")

In [45]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.transform_data()  # Fixed method name from transfomrm_data to transform_data
except Exception as e:
    raise e

[2025-09-02 21:47:29,197: INFO: common: yaml file: config\config.yml loaded successfully]
[2025-09-02 21:47:29,200: INFO: common: yaml file: params.yml loaded successfully]
[2025-09-02 21:47:29,204: INFO: common: yaml file: schema.yml loaded successfully]
[2025-09-02 21:47:29,200: INFO: common: yaml file: params.yml loaded successfully]
[2025-09-02 21:47:29,204: INFO: common: yaml file: schema.yml loaded successfully]
[2025-09-02 21:47:29,206: INFO: common: created directory at: artifacts]
[2025-09-02 21:47:29,208: INFO: common: created directory at: artifacts/data_transformation]
[2025-09-02 21:47:29,206: INFO: common: created directory at: artifacts]
[2025-09-02 21:47:29,208: INFO: common: created directory at: artifacts/data_transformation]
[2025-09-02 21:47:29,249: INFO: 2267750694: Splitted data into training and test sets]
[2025-09-02 21:47:29,250: INFO: 2267750694: Removed original date column after extracting temporal features]
[2025-09-02 21:47:29,251: INFO: 2267750694: Train 